In [1]:
// 使用须知
// 1. 用户需要有tmp表的写入权限，申请方法见：https://km.sankuai.com/page/411726480
// 2. 需要增加git权限，将你的公钥添加到https://dev.sankuai.com/code/home 又上角 SSH Key
// 3. 需要升级AFO的版本，（在提交机 /opt/meituan/versions 目录下 下载hdfs上的/user/hadoop-hdp/dushifeng/afo-clients-0.5.70.tar.gz 这个文件， ll /opt/meituan/versions/afo-clients-0.5.70/afo-tensorflow/afo-tensorflow-0.5.70/
// 然后执行 ln -snf /opt/meituan/versions/afo-clients-0.5.70/afo-tensorflow/afo-tensorflow-0.5.70 /opt/meituan/tensorflow-release 试一下
println("Start!!!")

Start!!!


In [22]:
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03_training.hadoop-waimaiadrd.training") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1000")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId

var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

Creating SparkSession


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@997bafe
spark: SparkSession = org.apache.spark.sql.SparkSession@59924b3f
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res21_15: String = "application_1669619394037_11648479"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

In [16]:
// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.spark.sql.{DataFrame, Row}

import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}


import scala.collection.immutable.Map

import scala.collection.mutable


defined function JSONObj2Map
import org.apache.hadoop.hive.ql.exec.UDF

import spark.implicits._

findIndex2: String => String = <function1>
res15_13: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
res15_14: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
defined function getDate
defined function getTimeSuffix
defined function dropTable
defined function executeSQL
defined function genDateList
defined function printCurTime
defined function showTopNFromSQL
d

In [4]:
// 配置区
val is_execute_sql = true
var table_suffix=getTimeSuffix()
var dt = "today"
// var befor1day = getDate(-1,dt)
// var fp_log_date_start=getDate(-4,dt)
// var fp_log_date_end=getDate(-2,dt)
// var predict_dt= getDate(-2,dt)
var befor1day = "20221210"
var fp_log_date_start="20221210"
var fp_log_date_end="20221210"
var predict_dt= "20221210"
var business="dsa_os_unify_model" //提取特征的业务线

is_execute_sql: Boolean = true
table_suffix: String = "20221214"
dt: String = "today"
befor1day: String = "20221210"
fp_log_date_start: String = "20221210"
fp_log_date_end: String = "20221210"
predict_dt: String = "20221210"
business: String = "dsa_os_unify_model"

In [5]:
// 获取featureID 和名字的对应关系
var featureId2Name=s"""
select distinct id, name
from origindb_ss.waimaiadtcpc_waimai_offline_feature_platform__wm_model_feature_map
where dt=${befor1day}
and status=0
"""
//构建 id到name映射的map
val featureId2NameMap = spark.sqlContext.sql(featureId2Name).na.drop().rdd.
      map(f => (f.getLong(0).toString, f.getString(1))).
      collectAsMap()


val feaArray = Array(
    "pvid string",
    "user_id bigint",
    "uuid string",
    "poi_id bigint",
    "request_time bigint",
    "rawbid double",
    "precisebid double",
    "demandbiztype bigint",
    "supplybiztype bigint",
    "is_expose bigint",
    "is_click bigint",
    "final_charge double",
    "is_order bigint",
    "revenue double",
    "rt_features string",
    "raw_features string",
    "modelname string",
    "business string",
    "params string",
    "dt string"
)

var logRawFeature=""

// 构建schema
import scala.collection.mutable.ArrayBuffer
val schemaArray = new ArrayBuffer[StructField]()
var columeNameSet= new mutable.HashSet[String]()
for (kv <- feaArray.map(_.split(" "))) {
    val groupKey = kv.head
    val groupType = kv.last
    columeNameSet.add(groupKey)
    logRawFeature+=groupKey+","
    if (groupType.startsWith("double")) {
        schemaArray += StructField(groupKey, DoubleType)
    } else if (groupType.startsWith("int")) {
        schemaArray += StructField(groupKey, IntegerType)
    } else if (groupType.startsWith("bigint")) {
        schemaArray += StructField(groupKey, LongType)
    } else {
        schemaArray += StructField(groupKey, StringType)
    }
}
logRawFeature=logRawFeature.stripSuffix(",")

var line=executeSQLLazy(s"""
select rt_features,raw_features
from mart_waimaiad_test.dsa_model_fp_os_rl_log
where dt = "${fp_log_date_end}"
and business ="${business}"
limit 1
""")
//scame 构建
var rt_map:Map[String, String] = Map()
var rawFeatureMap:Map[String, String] = Map()

line.collect().foreach(x=>{
    var rt_features=x.getString(0)
    var rawFeatureString= x.getString(1)
    println(rt_features)
    println(rawFeatureString)
    var rtMap =  JSONObj2Map(JSON.parseObject(rt_features))
    var rawFeature =  JSONObj2Map(JSON.parseObject(rawFeatureString))
    for(key<-rtMap.keys){
        //类型增加判断 todo:
        if(!columeNameSet.contains(featureId2NameMap.getOrElse(key, key))){
            // println(s"${key}:${rtMap.get(key).get}, ${rtMap.get(key).get.getClass.getSimpleName}")
            schemaArray+= StructField(featureId2NameMap.getOrElse(key, key), StringType)
            rt_map+=(key->featureId2NameMap.getOrElse(key, key))
            columeNameSet.add(featureId2NameMap.getOrElse(key, key))
        }
    }
    for(key<-rawFeature.keys){
        if(!columeNameSet.contains(key)){
            // println(s"${key}:${rawFeature.get(key).get}, ${rawFeature.get(key).get.getClass.getSimpleName}")
            schemaArray+= StructField(key, StringType)
            rawFeatureMap+=(key->key)
            columeNameSet.add(featureId2NameMap.getOrElse(key, key))
        }
    }
})

rdd at cmd4.sc:8

1 / 1

: 

In [ ]:
//将log表的特征列展开
var tmpFPExtendRTFeatureTableName=s"${user}_${business}_${System.currentTimeMillis()}"
var FPExtendRTFeaturetableName=s"tmp.dsa_model_fp_log_${user}_${business}_df"

var logData=s"""
select ${logRawFeature}
from mart_waimaiad_test.dsa_model_fp_os_rl_log
where dt between ${fp_log_date_start} and ${fp_log_date_end}
and business ="${business}" 
"""
println(logData)
var rawData = spark.sqlContext.sql(logData).rdd.flatMap{
      f =>
        var pvid=f.getAs[String]("pvid")
        var user_id=f.getAs[Long]("user_id")
        var uuid=f.getAs[String]("uuid")
        var poi_id=f.getAs[Long]("poi_id")
        var request_time=f.getAs[Long]("request_time")
        var rawbid=f.getAs[Double]("rawbid")
        var precisebid=f.getAs[Double]("precisebid")
        var demandbiztype=f.getAs[Long]("demandbiztype")
        var supplybiztype=f.getAs[Long]("supplybiztype")
        var is_expose=f.getAs[Long]("is_expose")
        var is_click=f.getAs[Long]("is_click")
        var final_charge=f.getAs[Double]("final_charge")
        var is_order=f.getAs[Long]("is_order")
        var revenue=f.getAs[Double]("revenue")
        var rt_features=f.getAs[String]("rt_features").replace("Infinity","-1.0")
        var raw_features=f.getAs[String]("raw_features").replace("Infinity","-1.0")
        var modelname=f.getAs[String]("modelname")
        var business=f.getAs[String]("business")
        var params=f.getAs[String]("params")
        var dt = f.getAs[String]("dt")
        var array =Array(pvid,user_id,uuid,poi_id,request_time,rawbid,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,rt_features,raw_features,modelname,business,params,dt)
        var rtFeaturesMap = JSONObj2Map(JSON.parseObject(rt_features))
        var rawFeaturesLawMap = JSONObj2Map(JSON.parseObject(raw_features))
        //当前只支持int，string和double的类型，复杂类型还未支持
        for(featureID<-rt_map.keys){
            if(rtFeaturesMap.contains(featureID)){
                array=array:+rtFeaturesMap.get(featureID).get
            }else{
                array=array:+ -0.678
            }
        }
        for(featureID<-rawFeatureMap.keys){
            if(rawFeatureMap.contains(featureID)){
                array=array:+rawFeaturesLawMap.get(featureID).get
            }else{
                array=array:+"unknown"
            }
        }
        var row =Row(array:_*)
        // var row = Row(pvid)
        List(row)
}
println(rawData.count())


In [ ]:
// 将数据写入到hive表
val tmpTable = spark.sqlContext.createDataFrame(rawData, StructType(schemaArray.toList))
tmpTable.toDF().registerTempTable(tmpFPExtendRTFeatureTableName)
var featureForCreateTable=s""
var allFeatures=""
for(i <- 0 to schemaArray.length-1){
    allFeatures=allFeatures+schemaArray(i).name+","
    if(i==schemaArray.length-1){
        featureForCreateTable=featureForCreateTable+schemaArray(i).name+" "+schemaArray(i).dataType.typeName
    }else{
        featureForCreateTable=featureForCreateTable+schemaArray(i).name+" "+schemaArray(i).dataType.typeName+","
    }
}
allFeatures=allFeatures.stripSuffix(",")
executeSQLLazy(s"""drop table if exists $FPExtendRTFeaturetableName""")
executeSQLLazy(s"""create table if not exists $FPExtendRTFeaturetableName(${featureForCreateTable}) STORED AS ORC""")
executeSQLLazy(s"""insert overwrite table ${FPExtendRTFeaturetableName} select ${allFeatures} from ${tmpFPExtendRTFeatureTableName}""")
// showTopNFromTableName(tmpFPExtendRTFeatureTableName,10)

In [6]:
var FPExtendRTFeaturetableName="tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df"
var processTable_high_min_max = createTableFromSQL(s"""
SELECT a.poi_id,
       a.dt,
       a.prod,
       begin.real_bln as budget ,
       end.real_bln as goal 
  FROM(
        SELECT poi_id,
               dt,
               prod,
               min(request_time) as budget_time,
               max(request_time) as goal_time
          FROM ${FPExtendRTFeaturetableName}
         GROUP BY poi_id,
                  dt,
                  prod
       )a
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               request_time
          from ${FPExtendRTFeaturetableName}
       )begin
    ON a.poi_id = begin.poi_id
       and a.dt = begin.dt
       and a.prod = begin.prod
       and a.budget_time = begin.request_time
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               request_time
          from ${FPExtendRTFeaturetableName}
       )end
    ON a.poi_id = end.poi_id
       and a.dt = end.dt
       and a.prod = end.prod
       and a.goal_time = end.request_time
""")

2022-12-14 09:44:51
执行 
 
SELECT a.poi_id,
       a.dt,
       a.prod,
       begin.real_bln as budget ,
       end.real_bln as goal 
  FROM(
        SELECT poi_id,
               dt,
               prod,
               min(request_time) as budget_time,
               max(request_time) as goal_time
          FROM tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df
         GROUP BY poi_id,
                  dt,
                  prod
       )a
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               request_time
          from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df
       )begin
    ON a.poi_id = begin.poi_id
       and a.dt = begin.dt
       and a.prod = begin.prod
       and a.budget_time = begin.request_time
  LEFT JOIN (
        SELECT poi_id,
               dt,
               prod,
               real_bln,
               request_time
          from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df
   

sql at cmd2.sc:136

2000 / 2000

sql at cmd2.sc:136

2000 / 2000

...

[Stage 5:======================================================>(199 + 1) / 200]

sql at cmd2.sc:136

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670982291125' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current


FPExtendRTFeaturetableName: String = "tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df"
processTable_high_min_max: String = "tmp.AFO_RL_1670982291125"

In [9]:
// high level
// ------------------------------------------
// ------------------------------------------
// 上层应该一个poi在一个prod上对应数据，不应该有pv和request_time
// 尝试新写法
var processTable_high = createTableFromSQL(s"""
SELECT
cast(full_log.dt as String) dt,
cast(full_log.poi_id as String) as poi_id,
cast(full_log.prod as float) as prod,

-- cast(max(user_id) as string) as user_id,
-- cast(max(uuid) as string) as uuid,
cast(max(cityid) as float) as cityid,

cast(if(max(ctr)>10, 0, max(ctr)) as float) as ctr,
cast(if(max(histctr)>10, 0, max(histctr)) as float) as histctr,

-- 商家的goal，最终计费和cpc约束
cast(sum(final_charge) as float) as final_charge,
cast(max(aimcpc) as float) as aimcpc,

-- 过去一天，过去同期四周
cast(if(max(impr)>1000, 0, max(cpc)) as float) as impr,
cast(if(max(cpc)>1000, 0, max(cpc)) as float) as cpc,
cast(if(max(revenue)>1000, 0, max(revenue)) as float) as revenue,
cast(if(max(charge)>1000, 0, max(charge)) as float) as charge,
cast(if(max(click)>1000, 0, max(click)) as float) as click,
cast(if(max(order_num)>1000, 0, max(order_num)) as float) as order_num,
cast(if(max(mt_butie)>1000, 0, max(mt_butie)) as float) as mt_butie,

cast(if(max(impr_before1week)>1000, 0, max(impr_before1week)) as float) as impr_before1week,
cast(if(max(click_before1week)>1000, 0, max(click_before1week)) as float) as click_before1week,
cast(if(max(cpc_before1week)>1000, 0, max(cpc_before1week)) as float) as cpc_before1week,
cast(if(max(order_num_before1week)>1000, 0, max(order_num_before1week)) as float) as order_num_before1week,
cast(if(max(revenue_before1week)>1000, 0, max(revenue_before1week)) as float) as revenue_before1week,
cast(if(max(mt_butie_before1week)>1000, 0, max(mt_butie_before1week)) as float) as mt_butie_before1week,
cast(if(max(charge_before1week)>1000, 0, max(charge_before1week)) as float) as charge_before1week,

cast(if(max(impr_before2week)>1000, 0, max(impr_before2week)) as float) as impr_before2week,
cast(if(max(click_before2week)>1000, 0, max(click_before2week)) as float) as click_before2week,
cast(if(max(cpc_before2week)>1000, 0, max(cpc_before2week)) as float) as cpc_before2week,
cast(if(max(revenue_before2week)>1000, 0, max(revenue_before2week)) as float) as revenue_before2week,
cast(if(max(charge_before2week)>1000, 0, max(charge_before2week)) as float) as charge_before2week,
cast(if(max(order_num_before2week)>1000, 0, max(order_num_before2week)) as float) as order_num_before2week,
cast(if(max(mt_butie_before2week)>1000, 0, max(mt_butie_before2week)) as float) as mt_butie_before2week,

cast(if(max(impr_before3week)>1000, 0, max(impr_before3week)) as float) as impr_before3week,
cast(if(max(cpc_before3week)>1000, 0, max(cpc_before3week)) as float) as cpc_before3week,
cast(if(max(click_before3week)>1000, 0, max(click_before3week)) as float) as click_before3week,
cast(if(max(order_num_before3week)>1000, 0, max(order_num_before3week)) as float) as order_num_before3week,
cast(if(max(mt_butie_before3week)>1000, 0, max(mt_butie_before3week)) as float) as mt_butie_before3week,
cast(if(max(revenue_before3week)>1000, 0, max(revenue_before3week)) as float) as revenue_before3week,
cast(if(max(charge_before3week)>1000, 0, max(charge_before3week)) as float) as charge_before3week,

cast(if(max(impr_before4week)>1000, 0, max(impr_before4week)) as float) as impr_before4week,
cast(if(max(charge_before4week)>1000, 0, max(charge_before4week)) as float) as charge_before4week,
cast(if(max(revenue_before4week)>1000, 0, max(revenue_before4week)) as float) as revenue_before4week,
cast(if(max(mt_butie_before4week)>1000, 0, max(mt_butie_before4week)) as float) as mt_butie_before4week,
cast(if(max(cpc_before4week)>1000, 0, max(cpc_before4week)) as float) as cpc_before4week,
cast(if(max(click_before4week)>1000, 0, max(click_before4week)) as float) as click_before4week,
cast(if(max(order_num_before4week)>1000, 0, max(order_num_before4week)) as float) as order_num_before4week,

cast(max(min_max_log.budget) as float) as budget,
cast(max(min_max_log.goal) as float) as goal


-- 不一定用到
-- cast(feacomp_ctr as float) as feacomp_ctr,
-- cast(ratioctr as float) as ratioctr,
-- cast(ratio_aimcpc as float) as ratio_aimcpc,
-- cast(winningscore_qs as float) as winningscore_qs,
-- cast(ratio_click as float) as ratio_click,
-- cast(ratioctr_exp as float) as ratioctr_exp

FROM ${FPExtendRTFeaturetableName} as full_log
LEFT JOIN(
    SELECT * FROM ${processTable_high_min_max}) min_max_log 
    on full_log.poi_id = min_max_log.poi_id
    and full_log.dt = min_max_log.dt
    and full_log.prod = min_max_log.prod
GROUP BY full_log.dt,
full_log.poi_id,
full_log.prod
""")

2022-12-14 09:54:50
执行 
 
SELECT
cast(full_log.dt as String) dt,
cast(full_log.poi_id as String) as poi_id,
cast(full_log.prod as float) as prod,

-- cast(max(user_id) as string) as user_id,
-- cast(max(uuid) as string) as uuid,
cast(max(cityid) as float) as cityid,

cast(if(max(ctr)>10, 0, max(ctr)) as float) as ctr,
cast(if(max(histctr)>10, 0, max(histctr)) as float) as histctr,

-- 商家的goal，最终计费和cpc约束
cast(sum(final_charge) as float) as final_charge,
cast(max(aimcpc) as float) as aimcpc,

-- 过去一天，过去同期四周
cast(if(max(impr)>1000, 0, max(cpc)) as float) as impr,
cast(if(max(cpc)>1000, 0, max(cpc)) as float) as cpc,
cast(if(max(revenue)>1000, 0, max(revenue)) as float) as revenue,
cast(if(max(charge)>1000, 0, max(charge)) as float) as charge,
cast(if(max(click)>1000, 0, max(click)) as float) as click,
cast(if(max(order_num)>1000, 0, max(order_num)) as float) as order_num,
cast(if(max(mt_butie)>1000, 0, max(mt_butie)) as float) as mt_butie,

cast(if(max(impr_before1week)>1000, 0, max(impr_

run at ThreadPoolExecutor.java:1142

200 / 200

sql at cmd2.sc:136

2000 / 2000

[Stage 9:====================================================>(1999 + 1) / 2000]

sql at cmd2.sc:136

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670982890627' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current


processTable_high: String = "tmp.AFO_RL_1670982890627"

In [10]:
// var trajectoryTableName_high=generateTrajectory(processTable_high,Array("poi_id","prod"),Array("dt"),Array("poi_id","prod"),"mart_waimaiad")
var trajectoryTableName_high=generateTrajectory(processTable_high,Array("poi_id","prod"),Array("dt"),Array("poi_id","prod"),"tmp")
// 生成了tmp.AFO_RL_1665159721593_trajectory

[Ljava.lang.String;@7e5911da
2022-12-14 10:03:06
执行 
 select * from tmp.AFO_RL_1670982890627 limit 1
2022-12-14 10:03:06
2022-12-14 10:03:06
执行 
 
        select t1.dt,t1.poi_id,t1.prod,t1.cityid,t1.ctr,t1.histctr,t1.final_charge,t1.aimcpc,t1.impr,t1.cpc,t1.revenue,t1.charge,t1.click,t1.order_num,t1.mt_butie,t1.impr_before1week,t1.click_before1week,t1.cpc_before1week,t1.order_num_before1week,t1.revenue_before1week,t1.mt_butie_before1week,t1.charge_before1week,t1.impr_before2week,t1.click_before2week,t1.cpc_before2week,t1.revenue_before2week,t1.charge_before2week,t1.order_num_before2week,t1.mt_butie_before2week,t1.impr_before3week,t1.cpc_before3week,t1.click_before3week,t1.order_num_before3week,t1.mt_butie_before3week,t1.revenue_before3week,t1.charge_before3week,t1.impr_before4week,t1.charge_before4week,t1.revenue_before4week,t1.mt_butie_before4week,t1.cpc_before4week,t1.click_before4week,t1.order_num_before4week,t1.budget,t1.goal,t1.rn,t2.dt as next_dt,t2.poi_id as next_poi_id,t2.prod 

sql at cmd2.sc:182

200 / 200

sql at cmd2.sc:182

200 / 200

...

[Stage 12:=============>(196 + 4) / 200][Stage 14:=============>(197 + 3) / 200]

sql at cmd2.sc:182

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670982890627_trajectory' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current


trajectoryTableName_high: String = "tmp.AFO_RL_1670982890627_trajectory"

In [11]:
showTopNFromSQL("""
select count(*) from tmp.AFO_RL_1670982890627_trajectory
""",10)

2022-12-14 10:06:16
2022-12-14 10:06:16
执行 
 
select count(*) from tmp.AFO_RL_1670982890627_trajectory

2022-12-14 10:06:16


show at cmd2.sc:96

200 / 200

[Stage 16:=====================================================>(199 + 1) / 200]

show at cmd2.sc:96

1 / 1

+--------+
|count(1)|
+--------+
|  347998|
+--------+

2022-12-14 10:06:19


In [17]:
// 由于含有array，手动生产
// high_level
var tableNameAfterTrajectory_high=createTableFromSQL(s"""
SELECT t1.dt,t1.poi_id,t1.prod,t1.reward,t1.is_terminal,
array(t1.ctr_dj,t1.ctr_bj,t1.ctr_ss,t1.ctr_push) as ctr,
array(t1.next_ctr_dj,t1.next_ctr_bj,t1.next_ctr_ss,t1.next_ctr_push) as next_ctr,
array(t1.histctr_dj,t1.histctr_bj,t1.histctr_ss,t1.histctr_push) as histctr,
array(t1.next_histctr_dj,t1.next_histctr_bj,t1.next_histctr_ss,t1.next_histctr_push) as next_histctr,
array(t1.final_charge_dj,t1.final_charge_bj,t1.final_charge_ss,t1.final_charge_push) as final_charge,
array(t1.next_final_charge_dj,t1.next_final_charge_bj,t1.next_final_charge_ss,t1.next_final_charge_push) as next_final_charge,
array(t1.aimcpc_dj,t1.aimcpc_bj,t1.aimcpc_ss,t1.aimcpc_push) as aimcpc,
array(t1.next_aimcpc_dj,t1.next_aimcpc_bj,t1.next_aimcpc_ss,t1.next_aimcpc_push) as next_aimcpc,
array(t1.impr_dj,t1.impr_bj,t1.impr_ss,t1.impr_push) as impr,
array(t1.next_impr_dj,t1.next_impr_bj,t1.next_impr_ss,t1.next_impr_push) as next_impr,
array(t1.cpc_dj,t1.cpc_bj,t1.cpc_ss,t1.cpc_push) as cpc,
array(t1.next_cpc_dj,t1.next_cpc_bj,t1.next_cpc_ss,t1.next_cpc_push) as next_cpc,
array(t1.revenue_dj,t1.revenue_bj,t1.revenue_ss,t1.revenue_push) as revenue,
array(t1.next_revenue_dj,t1.next_revenue_bj,t1.next_revenue_ss,t1.next_revenue_push) as next_revenue,
array(t1.charge_dj,t1.charge_bj,t1.charge_ss,t1.charge_push) as charge,
array(t1.next_charge_dj,t1.next_charge_bj,t1.next_charge_ss,t1.next_charge_push) as next_charge,
array(t1.click_dj,t1.click_bj,t1.click_ss,t1.click_push) as click,
array(t1.next_click_dj,t1.next_click_bj,t1.next_click_ss,t1.next_click_push) as next_click,
array(t1.order_num_dj,t1.order_num_bj,t1.order_num_ss,t1.order_num_push) as order_num,
array(t1.next_order_num_dj,t1.next_order_num_bj,t1.next_order_num_ss,t1.next_order_num_push) as next_order_num,
array(t1.mt_butie_dj,t1.mt_butie_bj,t1.mt_butie_ss,t1.mt_butie_push) as mt_butie,
array(t1.next_mt_butie_dj,t1.next_mt_butie_bj,t1.next_mt_butie_ss,t1.next_mt_butie_push) as next_mt_butie,
array(t1.budget_dj,t1.budget_bj,t1.budget_ss,t1.budget_push) as budget,
array(t1.next_budget_dj,t1.next_budget_bj,t1.next_budget_ss,t1.next_budget_push) as next_budget,
array(t1.goal_dj,t1.goal_bj,t1.goal_ss,t1.goal_push) as goal,
array(t1.next_goal_dj,t1.next_goal_bj,t1.next_goal_ss,t1.next_goal_push) as next_goal,
array(t1.impr_before1week_dj,t1.impr_before1week_bj,t1.impr_before1week_ss,t1.impr_before1week_push) as impr_before1week,
array(t1.next_impr_before1week_dj,t1.next_impr_before1week_bj,t1.next_impr_before1week_ss,t1.next_impr_before1week_push) as next_impr_before1week,
array(t1.charge_before1week_dj,t1.charge_before1week_bj,t1.charge_before1week_ss,t1.charge_before1week_push) as charge_before1week,
array(t1.next_charge_before1week_dj,t1.next_charge_before1week_bj,t1.next_charge_before1week_ss,t1.next_charge_before1week_push) as next_charge_before1week,
array(t1.click_before1week_dj,t1.click_before1week_bj,t1.click_before1week_ss,t1.click_before1week_push) as click_before1week,
array(t1.next_click_before1week_dj,t1.next_click_before1week_bj,t1.next_click_before1week_ss,t1.next_click_before1week_push) as next_click_before1week,
array(t1.cpc_before1week_dj,t1.cpc_before1week_bj,t1.cpc_before1week_ss,t1.cpc_before1week_push) as cpc_before1week,
array(t1.next_cpc_before1week_dj,t1.next_cpc_before1week_bj,t1.next_cpc_before1week_ss,t1.next_cpc_before1week_push) as next_cpc_before1week,
array(t1.order_num_before1week_dj,t1.order_num_before1week_bj,t1.order_num_before1week_ss,t1.order_num_before1week_push) as order_num_before1week,
array(t1.next_order_num_before1week_dj,t1.next_order_num_before1week_bj,t1.next_order_num_before1week_ss,t1.next_order_num_before1week_push) as next_order_num_before1week,
array(t1.revenue_before1week_dj,t1.revenue_before1week_bj,t1.revenue_before1week_ss,t1.revenue_before1week_push) as revenue_before1week,
array(t1.next_revenue_before1week_dj,t1.next_revenue_before1week_bj,t1.next_revenue_before1week_ss,t1.next_revenue_before1week_push) as next_revenue_before1week,
array(t1.mt_butie_before1week_dj,t1.mt_butie_before1week_bj,t1.mt_butie_before1week_ss,t1.mt_butie_before1week_push) as mt_butie_before1week,
array(t1.next_mt_butie_before1week_dj,t1.next_mt_butie_before1week_bj,t1.next_mt_butie_before1week_ss,t1.next_mt_butie_before1week_push) as next_mt_butie_before1week,
array(t1.impr_before2week_dj,t1.impr_before2week_bj,t1.impr_before2week_ss,t1.impr_before2week_push) as impr_before2week,
array(t1.next_impr_before2week_dj,t1.next_impr_before2week_bj,t1.next_impr_before2week_ss,t1.next_impr_before2week_push) as next_impr_before2week,
array(t1.charge_before2week_dj,t1.charge_before2week_bj,t1.charge_before2week_ss,t1.charge_before2week_push) as charge_before2week,
array(t1.next_charge_before2week_dj,t1.next_charge_before2week_bj,t1.next_charge_before2week_ss,t1.next_charge_before2week_push) as next_charge_before2week,
array(t1.click_before2week_dj,t1.click_before2week_bj,t1.click_before2week_ss,t1.click_before2week_push) as click_before2week,
array(t1.next_click_before2week_dj,t1.next_click_before2week_bj,t1.next_click_before2week_ss,t1.next_click_before2week_push) as next_click_before2week,
array(t1.cpc_before2week_dj,t1.cpc_before2week_bj,t1.cpc_before2week_ss,t1.cpc_before2week_push) as cpc_before2week,
array(t1.next_cpc_before2week_dj,t1.next_cpc_before2week_bj,t1.next_cpc_before2week_ss,t1.next_cpc_before2week_push) as next_cpc_before2week,
array(t1.order_num_before2week_dj,t1.order_num_before2week_bj,t1.order_num_before2week_ss,t1.order_num_before2week_push) as order_num_before2week,
array(t1.next_order_num_before2week_dj,t1.next_order_num_before2week_bj,t1.next_order_num_before2week_ss,t1.next_order_num_before2week_push) as next_order_num_before2week,
array(t1.revenue_before2week_dj,t1.revenue_before2week_bj,t1.revenue_before2week_ss,t1.revenue_before2week_push) as revenue_before2week,
array(t1.next_revenue_before2week_dj,t1.next_revenue_before2week_bj,t1.next_revenue_before2week_ss,t1.next_revenue_before2week_push) as next_revenue_before2week,
array(t1.mt_butie_before2week_dj,t1.mt_butie_before2week_bj,t1.mt_butie_before2week_ss,t1.mt_butie_before2week_push) as mt_butie_before2week,
array(t1.next_mt_butie_before2week_dj,t1.next_mt_butie_before2week_bj,t1.next_mt_butie_before2week_ss,t1.next_mt_butie_before2week_push) as next_mt_butie_before2week,
array(t1.impr_before3week_dj,t1.impr_before3week_bj,t1.impr_before3week_ss,t1.impr_before3week_push) as impr_before3week,
array(t1.next_impr_before3week_dj,t1.next_impr_before3week_bj,t1.next_impr_before3week_ss,t1.next_impr_before3week_push) as next_impr_before3week,
array(t1.charge_before3week_dj,t1.charge_before3week_bj,t1.charge_before3week_ss,t1.charge_before3week_push) as charge_before3week,
array(t1.next_charge_before3week_dj,t1.next_charge_before3week_bj,t1.next_charge_before3week_ss,t1.next_charge_before3week_push) as next_charge_before3week,
array(t1.click_before3week_dj,t1.click_before3week_bj,t1.click_before3week_ss,t1.click_before3week_push) as click_before3week,
array(t1.next_click_before3week_dj,t1.next_click_before3week_bj,t1.next_click_before3week_ss,t1.next_click_before3week_push) as next_click_before3week,
array(t1.cpc_before3week_dj,t1.cpc_before3week_bj,t1.cpc_before3week_ss,t1.cpc_before3week_push) as cpc_before3week,
array(t1.next_cpc_before3week_dj,t1.next_cpc_before3week_bj,t1.next_cpc_before3week_ss,t1.next_cpc_before3week_push) as next_cpc_before3week,
array(t1.order_num_before3week_dj,t1.order_num_before3week_bj,t1.order_num_before3week_ss,t1.order_num_before3week_push) as order_num_before3week,
array(t1.next_order_num_before3week_dj,t1.next_order_num_before3week_bj,t1.next_order_num_before3week_ss,t1.next_order_num_before3week_push) as next_order_num_before3week,
array(t1.revenue_before3week_dj,t1.revenue_before3week_bj,t1.revenue_before3week_ss,t1.revenue_before3week_push) as revenue_before3week,
array(t1.next_revenue_before3week_dj,t1.next_revenue_before3week_bj,t1.next_revenue_before3week_ss,t1.next_revenue_before3week_push) as next_revenue_before3week,
array(t1.mt_butie_before3week_dj,t1.mt_butie_before3week_bj,t1.mt_butie_before3week_ss,t1.mt_butie_before3week_push) as mt_butie_before3week,
array(t1.next_mt_butie_before3week_dj,t1.next_mt_butie_before3week_bj,t1.next_mt_butie_before3week_ss,t1.next_mt_butie_before3week_push) as next_mt_butie_before3week,
array(t1.impr_before4week_dj,t1.impr_before4week_bj,t1.impr_before4week_ss,t1.impr_before4week_push) as impr_before4week,
array(t1.next_impr_before4week_dj,t1.next_impr_before4week_bj,t1.next_impr_before4week_ss,t1.next_impr_before4week_push) as next_impr_before4week,
array(t1.charge_before4week_dj,t1.charge_before4week_bj,t1.charge_before4week_ss,t1.charge_before4week_push) as charge_before4week,
array(t1.next_charge_before4week_dj,t1.next_charge_before4week_bj,t1.next_charge_before4week_ss,t1.next_charge_before4week_push) as next_charge_before4week,
array(t1.click_before4week_dj,t1.click_before4week_bj,t1.click_before4week_ss,t1.click_before4week_push) as click_before4week,
array(t1.next_click_before4week_dj,t1.next_click_before4week_bj,t1.next_click_before4week_ss,t1.next_click_before4week_push) as next_click_before4week,
array(t1.cpc_before4week_dj,t1.cpc_before4week_bj,t1.cpc_before4week_ss,t1.cpc_before4week_push) as cpc_before4week,
array(t1.next_cpc_before4week_dj,t1.next_cpc_before4week_bj,t1.next_cpc_before4week_ss,t1.next_cpc_before4week_push) as next_cpc_before4week,
array(t1.order_num_before4week_dj,t1.order_num_before4week_bj,t1.order_num_before4week_ss,t1.order_num_before4week_push) as order_num_before4week,
array(t1.next_order_num_before4week_dj,t1.next_order_num_before4week_bj,t1.next_order_num_before4week_ss,t1.next_order_num_before4week_push) as next_order_num_before4week,
array(t1.revenue_before4week_dj,t1.revenue_before4week_bj,t1.revenue_before4week_ss,t1.revenue_before4week_push) as revenue_before4week,
array(t1.next_revenue_before4week_dj,t1.next_revenue_before4week_bj,t1.next_revenue_before4week_ss,t1.next_revenue_before4week_push) as next_revenue_before4week,
array(t1.mt_butie_before4week_dj,t1.mt_butie_before4week_bj,t1.mt_butie_before4week_ss,t1.mt_butie_before4week_push) as mt_butie_before4week,
array(t1.next_mt_butie_before4week_dj,t1.next_mt_butie_before4week_bj,t1.next_mt_butie_before4week_ss,t1.next_mt_butie_before4week_push) as next_mt_butie_before4week,
 CASE WHEN prod = 1.0 THEN array(dj,0,0,0)
      WHEN prod = 2.0 THEN array(0,bj,0,0)
      WHEN prod = 3.0 THEN array(0,0,ss,0)
      WHEN prod = 4.0 THEN array(0,0,0,push)
 END AS action
 FROM(
        select dt,
               poi_id,
               prod,
if( prod = 1, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_dj,
if( prod = 2, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_bj,
if( prod = 3, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_ss,
if( prod = 4, cast(coalesce(ctr, 0.0) as float) , 0) as ctr_push,
if( prod = 1, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_dj,
if( prod = 2, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_bj,
if( prod = 3, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_ss,
if( prod = 4, cast(coalesce(histctr, 0.0) as float) , 0) as histctr_push,
if( prod = 1, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_dj,
if( prod = 2, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_bj,
if( prod = 3, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_ss,
if( prod = 4, cast(coalesce(final_charge, 0.0) as float) , 0) as final_charge_push,
if( prod = 1, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_dj,
if( prod = 2, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_bj,
if( prod = 3, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_ss,
if( prod = 4, cast(coalesce(aimcpc, 0.0) as float) , 0) as aimcpc_push,
if( prod = 1, cast(coalesce(impr, 0.0) as float) , 0) as impr_dj,
if( prod = 2, cast(coalesce(impr, 0.0) as float) , 0) as impr_bj,
if( prod = 3, cast(coalesce(impr, 0.0) as float) , 0) as impr_ss,
if( prod = 4, cast(coalesce(impr, 0.0) as float) , 0) as impr_push,
if( prod = 1, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_dj,
if( prod = 2, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_bj,
if( prod = 3, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_ss,
if( prod = 4, cast(coalesce(cpc, 0.0) as float) , 0) as cpc_push,
if( prod = 1, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_dj,
if( prod = 2, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_bj,
if( prod = 3, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_ss,
if( prod = 4, cast(coalesce(revenue, 0.0) as float) , 0) as revenue_push,
if( prod = 1, cast(coalesce(charge, 0.0) as float) , 0) as charge_dj,
if( prod = 2, cast(coalesce(charge, 0.0) as float) , 0) as charge_bj,
if( prod = 3, cast(coalesce(charge, 0.0) as float) , 0) as charge_ss,
if( prod = 4, cast(coalesce(charge, 0.0) as float) , 0) as charge_push,
if( prod = 1, cast(coalesce(click, 0.0) as float) , 0) as click_dj,
if( prod = 2, cast(coalesce(click, 0.0) as float) , 0) as click_bj,
if( prod = 3, cast(coalesce(click, 0.0) as float) , 0) as click_ss,
if( prod = 4, cast(coalesce(click, 0.0) as float) , 0) as click_push,
if( prod = 1, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_dj,
if( prod = 2, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_bj,
if( prod = 3, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_ss,
if( prod = 4, cast(coalesce(order_num, 0.0) as float) , 0) as order_num_push,
if( prod = 1, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_dj,
if( prod = 2, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_bj,
if( prod = 3, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_ss,
if( prod = 4, cast(coalesce(mt_butie, 0.0) as float) , 0) as mt_butie_push,
if( prod = 1, cast(coalesce(budget, 0.0) as float) , 0) as budget_dj,
if( prod = 2, cast(coalesce(budget, 0.0) as float) , 0) as budget_bj,
if( prod = 3, cast(coalesce(budget, 0.0) as float) , 0) as budget_ss,
if( prod = 4, cast(coalesce(budget, 0.0) as float) , 0) as budget_push,
if( prod = 1, cast(coalesce(goal, 0.0) as float) , 0) as goal_dj,
if( prod = 2, cast(coalesce(goal, 0.0) as float) , 0) as goal_bj,
if( prod = 3, cast(coalesce(goal, 0.0) as float) , 0) as goal_ss,
if( prod = 4, cast(coalesce(goal, 0.0) as float) , 0) as goal_push,
if( prod = 1, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_dj,
if( prod = 2, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_bj,
if( prod = 3, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_ss,
if( prod = 4, cast(coalesce(impr_before1week, 0.0) as float) , 0) as impr_before1week_push,
if( prod = 1, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_dj,
if( prod = 2, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_bj,
if( prod = 3, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_ss,
if( prod = 4, cast(coalesce(charge_before1week, 0.0) as float) , 0) as charge_before1week_push,
if( prod = 1, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_dj,
if( prod = 2, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_bj,
if( prod = 3, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_ss,
if( prod = 4, cast(coalesce(click_before1week, 0.0) as float) , 0) as click_before1week_push,
if( prod = 1, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_dj,
if( prod = 2, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_bj,
if( prod = 3, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_ss,
if( prod = 4, cast(coalesce(cpc_before1week, 0.0) as float) , 0) as cpc_before1week_push,
if( prod = 1, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_dj,
if( prod = 2, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_bj,
if( prod = 3, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_ss,
if( prod = 4, cast(coalesce(order_num_before1week, 0.0) as float) , 0) as order_num_before1week_push,
if( prod = 1, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_dj,
if( prod = 2, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_bj,
if( prod = 3, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_ss,
if( prod = 4, cast(coalesce(revenue_before1week, 0.0) as float) , 0) as revenue_before1week_push,
if( prod = 1, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_dj,
if( prod = 2, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_bj,
if( prod = 3, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_ss,
if( prod = 4, cast(coalesce(mt_butie_before1week, 0.0) as float) , 0) as mt_butie_before1week_push,
if( prod = 1, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_dj,
if( prod = 2, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_bj,
if( prod = 3, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_ss,
if( prod = 4, cast(coalesce(impr_before2week, 0.0) as float) , 0) as impr_before2week_push,
if( prod = 1, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_dj,
if( prod = 2, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_bj,
if( prod = 3, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_ss,
if( prod = 4, cast(coalesce(charge_before2week, 0.0) as float) , 0) as charge_before2week_push,
if( prod = 1, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_dj,
if( prod = 2, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_bj,
if( prod = 3, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_ss,
if( prod = 4, cast(coalesce(click_before2week, 0.0) as float) , 0) as click_before2week_push,
if( prod = 1, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_dj,
if( prod = 2, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_bj,
if( prod = 3, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_ss,
if( prod = 4, cast(coalesce(cpc_before2week, 0.0) as float) , 0) as cpc_before2week_push,
if( prod = 1, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_dj,
if( prod = 2, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_bj,
if( prod = 3, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_ss,
if( prod = 4, cast(coalesce(order_num_before2week, 0.0) as float) , 0) as order_num_before2week_push,
if( prod = 1, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_dj,
if( prod = 2, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_bj,
if( prod = 3, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_ss,
if( prod = 4, cast(coalesce(revenue_before2week, 0.0) as float) , 0) as revenue_before2week_push,
if( prod = 1, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_dj,
if( prod = 2, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_bj,
if( prod = 3, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_ss,
if( prod = 4, cast(coalesce(mt_butie_before2week, 0.0) as float) , 0) as mt_butie_before2week_push,
if( prod = 1, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_dj,
if( prod = 2, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_bj,
if( prod = 3, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_ss,
if( prod = 4, cast(coalesce(impr_before3week, 0.0) as float) , 0) as impr_before3week_push,
if( prod = 1, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_dj,
if( prod = 2, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_bj,
if( prod = 3, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_ss,
if( prod = 4, cast(coalesce(charge_before3week, 0.0) as float) , 0) as charge_before3week_push,
if( prod = 1, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_dj,
if( prod = 2, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_bj,
if( prod = 3, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_ss,
if( prod = 4, cast(coalesce(click_before3week, 0.0) as float) , 0) as click_before3week_push,
if( prod = 1, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_dj,
if( prod = 2, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_bj,
if( prod = 3, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_ss,
if( prod = 4, cast(coalesce(cpc_before3week, 0.0) as float) , 0) as cpc_before3week_push,
if( prod = 1, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_dj,
if( prod = 2, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_bj,
if( prod = 3, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_ss,
if( prod = 4, cast(coalesce(order_num_before3week, 0.0) as float) , 0) as order_num_before3week_push,
if( prod = 1, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_dj,
if( prod = 2, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_bj,
if( prod = 3, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_ss,
if( prod = 4, cast(coalesce(revenue_before3week, 0.0) as float) , 0) as revenue_before3week_push,
if( prod = 1, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_dj,
if( prod = 2, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_bj,
if( prod = 3, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_ss,
if( prod = 4, cast(coalesce(mt_butie_before3week, 0.0) as float) , 0) as mt_butie_before3week_push,
if( prod = 1, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_dj,
if( prod = 2, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_bj,
if( prod = 3, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_ss,
if( prod = 4, cast(coalesce(impr_before4week, 0.0) as float) , 0) as impr_before4week_push,
if( prod = 1, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_dj,
if( prod = 2, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_bj,
if( prod = 3, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_ss,
if( prod = 4, cast(coalesce(charge_before4week, 0.0) as float) , 0) as charge_before4week_push,
if( prod = 1, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_dj,
if( prod = 2, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_bj,
if( prod = 3, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_ss,
if( prod = 4, cast(coalesce(click_before4week, 0.0) as float) , 0) as click_before4week_push,
if( prod = 1, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_dj,
if( prod = 2, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_bj,
if( prod = 3, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_ss,
if( prod = 4, cast(coalesce(cpc_before4week, 0.0) as float) , 0) as cpc_before4week_push,
if( prod = 1, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_dj,
if( prod = 2, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_bj,
if( prod = 3, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_ss,
if( prod = 4, cast(coalesce(order_num_before4week, 0.0) as float) , 0) as order_num_before4week_push,
if( prod = 1, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_dj,
if( prod = 2, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_bj,
if( prod = 3, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_ss,
if( prod = 4, cast(coalesce(revenue_before4week, 0.0) as float) , 0) as revenue_before4week_push,
if( prod = 1, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_dj,
if( prod = 2, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_bj,
if( prod = 3, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_ss,
if( prod = 4, cast(coalesce(mt_butie_before4week, 0.0) as float) , 0) as mt_butie_before4week_push,
if( prod = 1, cast(coalesce(next_ctr, 0.0) as float) , 0) as next_ctr_dj,
if( prod = 2, cast(coalesce(next_ctr, 0.0) as float) , 0) as next_ctr_bj,
if( prod = 3, cast(coalesce(next_ctr, 0.0) as float) , 0) as next_ctr_ss,
if( prod = 4, cast(coalesce(next_ctr, 0.0) as float) , 0) as next_ctr_push,
if( prod = 1, cast(coalesce(next_histctr, 0.0) as float) , 0) as next_histctr_dj,
if( prod = 2, cast(coalesce(next_histctr, 0.0) as float) , 0) as next_histctr_bj,
if( prod = 3, cast(coalesce(next_histctr, 0.0) as float) , 0) as next_histctr_ss,
if( prod = 4, cast(coalesce(next_histctr, 0.0) as float) , 0) as next_histctr_push,
if( prod = 1, cast(coalesce(next_final_charge, 0.0) as float) , 0) as next_final_charge_dj,
if( prod = 2, cast(coalesce(next_final_charge, 0.0) as float) , 0) as next_final_charge_bj,
if( prod = 3, cast(coalesce(next_final_charge, 0.0) as float) , 0) as next_final_charge_ss,
if( prod = 4, cast(coalesce(next_final_charge, 0.0) as float) , 0) as next_final_charge_push,
if( prod = 1, cast(coalesce(next_aimcpc, 0.0) as float) , 0) as next_aimcpc_dj,
if( prod = 2, cast(coalesce(next_aimcpc, 0.0) as float) , 0) as next_aimcpc_bj,
if( prod = 3, cast(coalesce(next_aimcpc, 0.0) as float) , 0) as next_aimcpc_ss,
if( prod = 4, cast(coalesce(next_aimcpc, 0.0) as float) , 0) as next_aimcpc_push,
if( prod = 1, cast(coalesce(next_impr, 0.0) as float) , 0) as next_impr_dj,
if( prod = 2, cast(coalesce(next_impr, 0.0) as float) , 0) as next_impr_bj,
if( prod = 3, cast(coalesce(next_impr, 0.0) as float) , 0) as next_impr_ss,
if( prod = 4, cast(coalesce(next_impr, 0.0) as float) , 0) as next_impr_push,
if( prod = 1, cast(coalesce(next_cpc, 0.0) as float) , 0) as next_cpc_dj,
if( prod = 2, cast(coalesce(next_cpc, 0.0) as float) , 0) as next_cpc_bj,
if( prod = 3, cast(coalesce(next_cpc, 0.0) as float) , 0) as next_cpc_ss,
if( prod = 4, cast(coalesce(next_cpc, 0.0) as float) , 0) as next_cpc_push,
if( prod = 1, cast(coalesce(next_revenue, 0.0) as float) , 0) as next_revenue_dj,
if( prod = 2, cast(coalesce(next_revenue, 0.0) as float) , 0) as next_revenue_bj,
if( prod = 3, cast(coalesce(next_revenue, 0.0) as float) , 0) as next_revenue_ss,
if( prod = 4, cast(coalesce(next_revenue, 0.0) as float) , 0) as next_revenue_push,
if( prod = 1, cast(coalesce(next_charge, 0.0) as float) , 0) as next_charge_dj,
if( prod = 2, cast(coalesce(next_charge, 0.0) as float) , 0) as next_charge_bj,
if( prod = 3, cast(coalesce(next_charge, 0.0) as float) , 0) as next_charge_ss,
if( prod = 4, cast(coalesce(next_charge, 0.0) as float) , 0) as next_charge_push,
if( prod = 1, cast(coalesce(next_click, 0.0) as float) , 0) as next_click_dj,
if( prod = 2, cast(coalesce(next_click, 0.0) as float) , 0) as next_click_bj,
if( prod = 3, cast(coalesce(next_click, 0.0) as float) , 0) as next_click_ss,
if( prod = 4, cast(coalesce(next_click, 0.0) as float) , 0) as next_click_push,
if( prod = 1, cast(coalesce(next_order_num, 0.0) as float) , 0) as next_order_num_dj,
if( prod = 2, cast(coalesce(next_order_num, 0.0) as float) , 0) as next_order_num_bj,
if( prod = 3, cast(coalesce(next_order_num, 0.0) as float) , 0) as next_order_num_ss,
if( prod = 4, cast(coalesce(next_order_num, 0.0) as float) , 0) as next_order_num_push,
if( prod = 1, cast(coalesce(next_mt_butie, 0.0) as float) , 0) as next_mt_butie_dj,
if( prod = 2, cast(coalesce(next_mt_butie, 0.0) as float) , 0) as next_mt_butie_bj,
if( prod = 3, cast(coalesce(next_mt_butie, 0.0) as float) , 0) as next_mt_butie_ss,
if( prod = 4, cast(coalesce(next_mt_butie, 0.0) as float) , 0) as next_mt_butie_push,
if( prod = 1, cast(coalesce(next_budget, 0.0) as float) , 0) as next_budget_dj,
if( prod = 2, cast(coalesce(next_budget, 0.0) as float) , 0) as next_budget_bj,
if( prod = 3, cast(coalesce(next_budget, 0.0) as float) , 0) as next_budget_ss,
if( prod = 4, cast(coalesce(next_budget, 0.0) as float) , 0) as next_budget_push,
if( prod = 1, cast(coalesce(next_goal, 0.0) as float) , 0) as next_goal_dj,
if( prod = 2, cast(coalesce(next_goal, 0.0) as float) , 0) as next_goal_bj,
if( prod = 3, cast(coalesce(next_goal, 0.0) as float) , 0) as next_goal_ss,
if( prod = 4, cast(coalesce(next_goal, 0.0) as float) , 0) as next_goal_push,
if( prod = 1, cast(coalesce(next_impr_before1week, 0.0) as float) , 0) as next_impr_before1week_dj,
if( prod = 2, cast(coalesce(next_impr_before1week, 0.0) as float) , 0) as next_impr_before1week_bj,
if( prod = 3, cast(coalesce(next_impr_before1week, 0.0) as float) , 0) as next_impr_before1week_ss,
if( prod = 4, cast(coalesce(next_impr_before1week, 0.0) as float) , 0) as next_impr_before1week_push,
if( prod = 1, cast(coalesce(next_charge_before1week, 0.0) as float) , 0) as next_charge_before1week_dj,
if( prod = 2, cast(coalesce(next_charge_before1week, 0.0) as float) , 0) as next_charge_before1week_bj,
if( prod = 3, cast(coalesce(next_charge_before1week, 0.0) as float) , 0) as next_charge_before1week_ss,
if( prod = 4, cast(coalesce(next_charge_before1week, 0.0) as float) , 0) as next_charge_before1week_push,
if( prod = 1, cast(coalesce(next_click_before1week, 0.0) as float) , 0) as next_click_before1week_dj,
if( prod = 2, cast(coalesce(next_click_before1week, 0.0) as float) , 0) as next_click_before1week_bj,
if( prod = 3, cast(coalesce(next_click_before1week, 0.0) as float) , 0) as next_click_before1week_ss,
if( prod = 4, cast(coalesce(next_click_before1week, 0.0) as float) , 0) as next_click_before1week_push,
if( prod = 1, cast(coalesce(next_cpc_before1week, 0.0) as float) , 0) as next_cpc_before1week_dj,
if( prod = 2, cast(coalesce(next_cpc_before1week, 0.0) as float) , 0) as next_cpc_before1week_bj,
if( prod = 3, cast(coalesce(next_cpc_before1week, 0.0) as float) , 0) as next_cpc_before1week_ss,
if( prod = 4, cast(coalesce(next_cpc_before1week, 0.0) as float) , 0) as next_cpc_before1week_push,
if( prod = 1, cast(coalesce(next_order_num_before1week, 0.0) as float) , 0) as next_order_num_before1week_dj,
if( prod = 2, cast(coalesce(next_order_num_before1week, 0.0) as float) , 0) as next_order_num_before1week_bj,
if( prod = 3, cast(coalesce(next_order_num_before1week, 0.0) as float) , 0) as next_order_num_before1week_ss,
if( prod = 4, cast(coalesce(next_order_num_before1week, 0.0) as float) , 0) as next_order_num_before1week_push,
if( prod = 1, cast(coalesce(next_revenue_before1week, 0.0) as float) , 0) as next_revenue_before1week_dj,
if( prod = 2, cast(coalesce(next_revenue_before1week, 0.0) as float) , 0) as next_revenue_before1week_bj,
if( prod = 3, cast(coalesce(next_revenue_before1week, 0.0) as float) , 0) as next_revenue_before1week_ss,
if( prod = 4, cast(coalesce(next_revenue_before1week, 0.0) as float) , 0) as next_revenue_before1week_push,
if( prod = 1, cast(coalesce(next_mt_butie_before1week, 0.0) as float) , 0) as next_mt_butie_before1week_dj,
if( prod = 2, cast(coalesce(next_mt_butie_before1week, 0.0) as float) , 0) as next_mt_butie_before1week_bj,
if( prod = 3, cast(coalesce(next_mt_butie_before1week, 0.0) as float) , 0) as next_mt_butie_before1week_ss,
if( prod = 4, cast(coalesce(next_mt_butie_before1week, 0.0) as float) , 0) as next_mt_butie_before1week_push,
if( prod = 1, cast(coalesce(next_impr_before2week, 0.0) as float) , 0) as next_impr_before2week_dj,
if( prod = 2, cast(coalesce(next_impr_before2week, 0.0) as float) , 0) as next_impr_before2week_bj,
if( prod = 3, cast(coalesce(next_impr_before2week, 0.0) as float) , 0) as next_impr_before2week_ss,
if( prod = 4, cast(coalesce(next_impr_before2week, 0.0) as float) , 0) as next_impr_before2week_push,
if( prod = 1, cast(coalesce(next_charge_before2week, 0.0) as float) , 0) as next_charge_before2week_dj,
if( prod = 2, cast(coalesce(next_charge_before2week, 0.0) as float) , 0) as next_charge_before2week_bj,
if( prod = 3, cast(coalesce(next_charge_before2week, 0.0) as float) , 0) as next_charge_before2week_ss,
if( prod = 4, cast(coalesce(next_charge_before2week, 0.0) as float) , 0) as next_charge_before2week_push,
if( prod = 1, cast(coalesce(next_click_before2week, 0.0) as float) , 0) as next_click_before2week_dj,
if( prod = 2, cast(coalesce(next_click_before2week, 0.0) as float) , 0) as next_click_before2week_bj,
if( prod = 3, cast(coalesce(next_click_before2week, 0.0) as float) , 0) as next_click_before2week_ss,
if( prod = 4, cast(coalesce(next_click_before2week, 0.0) as float) , 0) as next_click_before2week_push,
if( prod = 1, cast(coalesce(next_cpc_before2week, 0.0) as float) , 0) as next_cpc_before2week_dj,
if( prod = 2, cast(coalesce(next_cpc_before2week, 0.0) as float) , 0) as next_cpc_before2week_bj,
if( prod = 3, cast(coalesce(next_cpc_before2week, 0.0) as float) , 0) as next_cpc_before2week_ss,
if( prod = 4, cast(coalesce(next_cpc_before2week, 0.0) as float) , 0) as next_cpc_before2week_push,
if( prod = 1, cast(coalesce(next_order_num_before2week, 0.0) as float) , 0) as next_order_num_before2week_dj,
if( prod = 2, cast(coalesce(next_order_num_before2week, 0.0) as float) , 0) as next_order_num_before2week_bj,
if( prod = 3, cast(coalesce(next_order_num_before2week, 0.0) as float) , 0) as next_order_num_before2week_ss,
if( prod = 4, cast(coalesce(next_order_num_before2week, 0.0) as float) , 0) as next_order_num_before2week_push,
if( prod = 1, cast(coalesce(next_revenue_before2week, 0.0) as float) , 0) as next_revenue_before2week_dj,
if( prod = 2, cast(coalesce(next_revenue_before2week, 0.0) as float) , 0) as next_revenue_before2week_bj,
if( prod = 3, cast(coalesce(next_revenue_before2week, 0.0) as float) , 0) as next_revenue_before2week_ss,
if( prod = 4, cast(coalesce(next_revenue_before2week, 0.0) as float) , 0) as next_revenue_before2week_push,
if( prod = 1, cast(coalesce(next_mt_butie_before2week, 0.0) as float) , 0) as next_mt_butie_before2week_dj,
if( prod = 2, cast(coalesce(next_mt_butie_before2week, 0.0) as float) , 0) as next_mt_butie_before2week_bj,
if( prod = 3, cast(coalesce(next_mt_butie_before2week, 0.0) as float) , 0) as next_mt_butie_before2week_ss,
if( prod = 4, cast(coalesce(next_mt_butie_before2week, 0.0) as float) , 0) as next_mt_butie_before2week_push,
if( prod = 1, cast(coalesce(next_impr_before3week, 0.0) as float) , 0) as next_impr_before3week_dj,
if( prod = 2, cast(coalesce(next_impr_before3week, 0.0) as float) , 0) as next_impr_before3week_bj,
if( prod = 3, cast(coalesce(next_impr_before3week, 0.0) as float) , 0) as next_impr_before3week_ss,
if( prod = 4, cast(coalesce(next_impr_before3week, 0.0) as float) , 0) as next_impr_before3week_push,
if( prod = 1, cast(coalesce(next_charge_before3week, 0.0) as float) , 0) as next_charge_before3week_dj,
if( prod = 2, cast(coalesce(next_charge_before3week, 0.0) as float) , 0) as next_charge_before3week_bj,
if( prod = 3, cast(coalesce(next_charge_before3week, 0.0) as float) , 0) as next_charge_before3week_ss,
if( prod = 4, cast(coalesce(next_charge_before3week, 0.0) as float) , 0) as next_charge_before3week_push,
if( prod = 1, cast(coalesce(next_click_before3week, 0.0) as float) , 0) as next_click_before3week_dj,
if( prod = 2, cast(coalesce(next_click_before3week, 0.0) as float) , 0) as next_click_before3week_bj,
if( prod = 3, cast(coalesce(next_click_before3week, 0.0) as float) , 0) as next_click_before3week_ss,
if( prod = 4, cast(coalesce(next_click_before3week, 0.0) as float) , 0) as next_click_before3week_push,
if( prod = 1, cast(coalesce(next_cpc_before3week, 0.0) as float) , 0) as next_cpc_before3week_dj,
if( prod = 2, cast(coalesce(next_cpc_before3week, 0.0) as float) , 0) as next_cpc_before3week_bj,
if( prod = 3, cast(coalesce(next_cpc_before3week, 0.0) as float) , 0) as next_cpc_before3week_ss,
if( prod = 4, cast(coalesce(next_cpc_before3week, 0.0) as float) , 0) as next_cpc_before3week_push,
if( prod = 1, cast(coalesce(next_order_num_before3week, 0.0) as float) , 0) as next_order_num_before3week_dj,
if( prod = 2, cast(coalesce(next_order_num_before3week, 0.0) as float) , 0) as next_order_num_before3week_bj,
if( prod = 3, cast(coalesce(next_order_num_before3week, 0.0) as float) , 0) as next_order_num_before3week_ss,
if( prod = 4, cast(coalesce(next_order_num_before3week, 0.0) as float) , 0) as next_order_num_before3week_push,
if( prod = 1, cast(coalesce(next_revenue_before3week, 0.0) as float) , 0) as next_revenue_before3week_dj,
if( prod = 2, cast(coalesce(next_revenue_before3week, 0.0) as float) , 0) as next_revenue_before3week_bj,
if( prod = 3, cast(coalesce(next_revenue_before3week, 0.0) as float) , 0) as next_revenue_before3week_ss,
if( prod = 4, cast(coalesce(next_revenue_before3week, 0.0) as float) , 0) as next_revenue_before3week_push,
if( prod = 1, cast(coalesce(next_mt_butie_before3week, 0.0) as float) , 0) as next_mt_butie_before3week_dj,
if( prod = 2, cast(coalesce(next_mt_butie_before3week, 0.0) as float) , 0) as next_mt_butie_before3week_bj,
if( prod = 3, cast(coalesce(next_mt_butie_before3week, 0.0) as float) , 0) as next_mt_butie_before3week_ss,
if( prod = 4, cast(coalesce(next_mt_butie_before3week, 0.0) as float) , 0) as next_mt_butie_before3week_push,
if( prod = 1, cast(coalesce(next_impr_before4week, 0.0) as float) , 0) as next_impr_before4week_dj,
if( prod = 2, cast(coalesce(next_impr_before4week, 0.0) as float) , 0) as next_impr_before4week_bj,
if( prod = 3, cast(coalesce(next_impr_before4week, 0.0) as float) , 0) as next_impr_before4week_ss,
if( prod = 4, cast(coalesce(next_impr_before4week, 0.0) as float) , 0) as next_impr_before4week_push,
if( prod = 1, cast(coalesce(next_charge_before4week, 0.0) as float) , 0) as next_charge_before4week_dj,
if( prod = 2, cast(coalesce(next_charge_before4week, 0.0) as float) , 0) as next_charge_before4week_bj,
if( prod = 3, cast(coalesce(next_charge_before4week, 0.0) as float) , 0) as next_charge_before4week_ss,
if( prod = 4, cast(coalesce(next_charge_before4week, 0.0) as float) , 0) as next_charge_before4week_push,
if( prod = 1, cast(coalesce(next_click_before4week, 0.0) as float) , 0) as next_click_before4week_dj,
if( prod = 2, cast(coalesce(next_click_before4week, 0.0) as float) , 0) as next_click_before4week_bj,
if( prod = 3, cast(coalesce(next_click_before4week, 0.0) as float) , 0) as next_click_before4week_ss,
if( prod = 4, cast(coalesce(next_click_before4week, 0.0) as float) , 0) as next_click_before4week_push,
if( prod = 1, cast(coalesce(next_cpc_before4week, 0.0) as float) , 0) as next_cpc_before4week_dj,
if( prod = 2, cast(coalesce(next_cpc_before4week, 0.0) as float) , 0) as next_cpc_before4week_bj,
if( prod = 3, cast(coalesce(next_cpc_before4week, 0.0) as float) , 0) as next_cpc_before4week_ss,
if( prod = 4, cast(coalesce(next_cpc_before4week, 0.0) as float) , 0) as next_cpc_before4week_push,
if( prod = 1, cast(coalesce(next_order_num_before4week, 0.0) as float) , 0) as next_order_num_before4week_dj,
if( prod = 2, cast(coalesce(next_order_num_before4week, 0.0) as float) , 0) as next_order_num_before4week_bj,
if( prod = 3, cast(coalesce(next_order_num_before4week, 0.0) as float) , 0) as next_order_num_before4week_ss,
if( prod = 4, cast(coalesce(next_order_num_before4week, 0.0) as float) , 0) as next_order_num_before4week_push,
if( prod = 1, cast(coalesce(next_revenue_before4week, 0.0) as float) , 0) as next_revenue_before4week_dj,
if( prod = 2, cast(coalesce(next_revenue_before4week, 0.0) as float) , 0) as next_revenue_before4week_bj,
if( prod = 3, cast(coalesce(next_revenue_before4week, 0.0) as float) , 0) as next_revenue_before4week_ss,
if( prod = 4, cast(coalesce(next_revenue_before4week, 0.0) as float) , 0) as next_revenue_before4week_push,
if( prod = 1, cast(coalesce(next_mt_butie_before4week, 0.0) as float) , 0) as next_mt_butie_before4week_dj,
if( prod = 2, cast(coalesce(next_mt_butie_before4week, 0.0) as float) , 0) as next_mt_butie_before4week_bj,
if( prod = 3, cast(coalesce(next_mt_butie_before4week, 0.0) as float) , 0) as next_mt_butie_before4week_ss,
if( prod = 4, cast(coalesce(next_mt_butie_before4week, 0.0) as float) , 0) as next_mt_butie_before4week_push,

          cast(coalesce(-(goal / budget), 0.0) as float) as reward,
          cast(if(next_dt is null, 1.0, 0.0) as float) as is_terminal
               
          from ${trajectoryTableName_high}
       )t1
  left join(
        SELECT poi_id,
       dt,
       CASE WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.0 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.1 THEN 0
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.1 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.2 THEN 1
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.2 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.3 THEN 2
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.3 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.4 THEN 3
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.4 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.5 THEN 4
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.5 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.6 THEN 5
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.6 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.7 THEN 6
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.7 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.8 THEN 7
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.8 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.9 THEN 8
            WHEN jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.9 AND jifei_dj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 1.0 THEN 9           
            ELSE 10
        END as dj,
       
       CASE WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.0 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.1 THEN 0
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.1 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.2 THEN 1
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.2 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.3 THEN 2
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.3 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.4 THEN 3
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.4 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.5 THEN 4
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.5 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.6 THEN 5
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.6 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.7 THEN 6
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.7 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.8 THEN 7
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.8 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.9 THEN 8
            WHEN jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.9 AND jifei_bj/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 1.0 THEN 9           
            ELSE 10
        END as bj,
       
       CASE WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.0 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.1 THEN 0
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.1 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.2 THEN 1
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.2 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.3 THEN 2
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.3 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.4 THEN 3
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.4 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.5 THEN 4
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.5 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.6 THEN 5
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.6 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.7 THEN 6
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.7 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.8 THEN 7
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.8 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.9 THEN 8
            WHEN jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.9 AND jifei_ss/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 1.0 THEN 9           
            ELSE 10
        END as ss,
       
       CASE WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.0 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.1 THEN 0
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.1 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.2 THEN 1
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.2 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.3 THEN 2
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.3 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.4 THEN 3
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.4 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.5 THEN 4
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.5 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.6 THEN 5
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.6 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.7 THEN 6
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.7 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.8 THEN 7
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.8 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 0.9 THEN 8
            WHEN jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) >= 0.9 AND jifei_push/(jifei_dj+jifei_bj+jifei_ss+jifei_push) < 1.0 THEN 9
            ELSE 10
        END as push
  FROM (
        select poi_id,
               dt,
               max(jifei_dj) as jifei_dj,
               max(jifei_bj) as jifei_bj,
               max(jifei_ss) as jifei_ss,
               max(jifei_push) as jifei_push from(
                SELECT poi_id,
                       dt,
                       if(prod=1.0, final_charge, 0) as jifei_dj,
                       if(prod=2.0, final_charge, 0) as jifei_bj,
                       if(prod=3.0, final_charge, 0) as jifei_ss,
                       if(prod=4.0, final_charge, 0) as jifei_push
                  FROM ${processTable_high}
               )
         GROUP BY poi_id,
                  dt
       )
     )t2
    ON t1.dt = t2.dt
   AND t1.poi_id = t2.poi_id
""")


2022-12-14 10:28:47
执行 
 
SELECT t1.dt,t1.poi_id,t1.prod,t1.reward,t1.is_terminal,
array(t1.ctr_dj,t1.ctr_bj,t1.ctr_ss,t1.ctr_push) as ctr,
array(t1.next_ctr_dj,t1.next_ctr_bj,t1.next_ctr_ss,t1.next_ctr_push) as next_ctr,
array(t1.histctr_dj,t1.histctr_bj,t1.histctr_ss,t1.histctr_push) as histctr,
array(t1.next_histctr_dj,t1.next_histctr_bj,t1.next_histctr_ss,t1.next_histctr_push) as next_histctr,
array(t1.final_charge_dj,t1.final_charge_bj,t1.final_charge_ss,t1.final_charge_push) as final_charge,
array(t1.next_final_charge_dj,t1.next_final_charge_bj,t1.next_final_charge_ss,t1.next_final_charge_push) as next_final_charge,
array(t1.aimcpc_dj,t1.aimcpc_bj,t1.aimcpc_ss,t1.aimcpc_push) as aimcpc,
array(t1.next_aimcpc_dj,t1.next_aimcpc_bj,t1.next_aimcpc_ss,t1.next_aimcpc_push) as next_aimcpc,
array(t1.impr_dj,t1.impr_bj,t1.impr_ss,t1.impr_push) as impr,
array(t1.next_impr_dj,t1.next_impr_bj,t1.next_impr_ss,t1.next_impr_push) as next_impr,
array(t1.cpc_dj,t1.cpc_bj,t1.cpc_ss,t1.cpc_push) as 

run at ThreadPoolExecutor.java:1142

200 / 200

[Stage 18:=====================================================>(199 + 1) / 200]

run at ThreadPoolExecutor.java:1142

200 / 200

sql at cmd15.sc:66

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670984927376' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2022-12-14 10:29:22


tableNameAfterTrajectory_high: String = "tmp.AFO_RL_1670984927376"

In [18]:
// var tableNameAfterTrajectory_high = "tmp.AFO_RL_1665933624473"
var tableNameAfterTrajectory = tableNameAfterTrajectory_high

tableNameAfterTrajectory: String = "tmp.AFO_RL_1670984927376"

In [19]:
// var tableNameAfterTrajectory = "mart_waimaiad.AFO_RL_1664328251754"
var tmpTrainTable = createTableFromSQL(s"""
select *
from ${tableNameAfterTrajectory}
where dt>=${fp_log_date_start}
and dt<= ${fp_log_date_end}
""")
//因为hive直读当前需要分区表，所以创建一个分区表
var finalTrainTable = createPTTableFromTableWithSpace(tmpTrainTable,"mart_waimaiad")
var tmpPredictTable = createTableFromSQL(s"""
select *
from ${tableNameAfterTrajectory}
where dt = ${predict_dt}
-- and abs(hash(pvid)) % 100 < 10
""")
// 生成评估表
var finalPredictTable = createPTTableFromTableWithSpace(tmpPredictTable,"mart_waimaiad") //正式使用请按照自己的逻辑生成，表的主键是pvid，poi_id
// 预测生成的表结构为  str(prediction['pvid']), str(prediction['poi_id']), str(prediction['qvalue'][0]), str(prediction['action'][0]),str(prediction['cur_action']))

2022-12-14 10:30:20
执行 
 
select *
from tmp.AFO_RL_1670984927376
where dt>=20221210
and dt<= 20221210

2022-12-14 10:30:20
2022-12-14 10:30:20
执行 
 drop table if exists tmp.AFO_RL_1670985020576
2022-12-14 10:30:20
2022-12-14 10:30:20
执行 
 create table if not exists tmp.AFO_RL_1670985020576(dt String,poi_id String,prod Float,reward Float,is_terminal Float,ctr Array<Float>,next_ctr Array<Float>,histctr Array<Float>,next_histctr Array<Float>,final_charge Array<Float>,next_final_charge Array<Float>,aimcpc Array<Float>,next_aimcpc Array<Float>,impr Array<Float>,next_impr Array<Float>,cpc Array<Float>,next_cpc Array<Float>,revenue Array<Float>,next_revenue Array<Float>,charge Array<Float>,next_charge Array<Float>,click Array<Float>,next_click Array<Float>,order_num Array<Float>,next_order_num Array<Float>,mt_butie Array<Float>,next_mt_butie Array<Float>,budget Array<Float>,next_budget Array<Float>,goal Array<Float>,next_goal Array<Float>,impr_before1week Array<Float>,next_impr_before1week Ar

sql at cmd15.sc:66

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670985020576' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2022-12-14 10:30:26
2022-12-14 10:30:26
执行 
 select * from tmp.AFO_RL_1670985020576
2022-12-14 10:30:26
2022-12-14 10:30:26
执行 
 drop table if exists mart_waimaiad.AFO_RL_1670985026703
2022-12-14 10:30:26
2022-12-14 10:30:26
执行 
 create table if not exists mart_waimaiad.AFO_RL_1670985026703(dt String,poi_id String,prod Float,reward Float,is_terminal Float,ctr Array<Float>,next_ctr Array<Float>,histctr Array<Float>,next_histctr Array<Float>,final_charge Array<Float>,next_final_charge Array<Float>,aimcpc Array<Float>,next_aimcpc Array<Float>,impr Array<Float>,next_impr Array<Float>,cpc Array<Float>,next_cpc Array<Float>,revenue Array<Float>,next_revenue Array<Float>,charge Array<Float>,next_charge Array<Float>,click Array<Float>,next_click Array<Float>,order_num Array<Float>,next_order_num Array<Float>,mt_butie Array<Float>,next_mt_butie

sql at cmd15.sc:236

...

200 / 200

Moved: 'viewfs://hadoop-meituan/user/hive/warehouse/tmp.db/afo_rl_1670985030984' to trash at: hdfs://dfsrouter/user/hadoop-hmart-waimaiad/.Trash/Current
2022-12-14 10:30:38
2022-12-14 10:30:38
执行 
 select * from tmp.AFO_RL_1670985030984
2022-12-14 10:30:38
2022-12-14 10:30:38
执行 
 drop table if exists mart_waimaiad.AFO_RL_1670985038253
2022-12-14 10:30:38
2022-12-14 10:30:38
执行 
 create table if not exists mart_waimaiad.AFO_RL_1670985038253(dt String,poi_id String,prod Float,reward Float,is_terminal Float,ctr Array<Float>,next_ctr Array<Float>,histctr Array<Float>,next_histctr Array<Float>,final_charge Array<Float>,next_final_charge Array<Float>,aimcpc Array<Float>,next_aimcpc Array<Float>,impr Array<Float>,next_impr Array<Float>,cpc Array<Float>,next_cpc Array<Float>,revenue Array<Float>,next_revenue Array<Float>,charge Array<Float>,next_charge Array<Float>,click Array<Float>,next_click Array<Float>,order_num Array<Float>,next_order_num Array<Float>,mt_butie Array<Float>,next_mt_butie

sql at cmd15.sc:236

200 / 200

tmpTrainTable: String = "tmp.AFO_RL_1670985020576"
finalTrainTable: String = "mart_waimaiad.AFO_RL_1670985026703"
tmpPredictTable: String = "tmp.AFO_RL_1670985030984"
finalPredictTable: String = "mart_waimaiad.AFO_RL_1670985038253"

In [ ]:
// ##################################################################
// #                                                                #
// #                                                                #
// #                         DeepRL-AFO                             #
// #                                                                #
// #                          Start...                              #
// ##################################################################

In [20]:
// var finalTrainTable = "mart_waimaiad.AFO_RL_1665989605657"
// var finalPredictTable = "mart_waimaiad.AFO_RL_1665989628626"
finalTrainTable

res19: String = "mart_waimaiad.AFO_RL_1670985026703"

In [ ]:
// 需要修改的地方：训练配置区域
var train_table_name=finalTrainTable//训练表名
var trainDt=s"""pt=${befor1day}"""
var predictDt=s"""pt=${befor1day}"""
// var trainDt=s"""pt=20221117"""
// var predictDt=s"""pt=20221117"""
var predict_table_name=finalPredictTable //评估表名 理论上不一样，当前就直接用了，正式使用的时候请使用另外一个表
// var predict_table_name=finalTrainTable
// var cur_state_dynamic_fea="rawbid,precisebid,demandbiztype,supplybiztype,is_expose,is_click,final_charge,is_order,revenue,unused,click,user_tag_pref_click_3day,hour,user_ord_num_weekdays,charge_before2week,order_num_before2week,impr_before1week,feacomp_cvr,click_before1week,lowerk,real_cpc,real_csm,revenue_before3week,cpc_before1week,charge_before3week,winningscore_comp12,impr,real_cst_bdgt,real_aBln,feacomp_ctr,real_expose,real_revenue,revenue_before2week,ratioctr,real_price,lowerbid,ratio_aimcpc,cpc_before4week,order_num_before3week,histctr,mt_butie_before2week,mt_butie_before3week,real_budget,order_num_before4week,cpc,order_num_before1week,revenue_before1week,mt_butie,cpc_before2week,cpc_before3week,aimcpc,mt_butie_before1week,real_csm_bln,click_before3week,charge_before1week,click_before4week,click_before2week,real_click,winningscore_qs,cityid,ratio_click,prod,ratioctr_exp,upperk,charge,revenue_before4week,ctr,order_num,impr_before4week,impr_before2week,mt_butie_before4week,real_order,ratio_cpc,impr_before3week,charge_before4week,real_bln".toLowerCase() //RL 状态特征
// var next_state_dynamic_fea="next_rawbid,next_precisebid,next_demandbiztype,next_supplybiztype,next_is_expose,next_is_click,next_final_charge,next_is_order,next_revenue,next_unused,next_click,next_user_tag_pref_click_3day,next_hour,next_user_ord_num_weekdays,next_charge_before2week,next_order_num_before2week,next_impr_before1week,next_feacomp_cvr,next_click_before1week,next_lowerk,next_real_cpc,next_real_csm,next_revenue_before3week,next_cpc_before1week,next_charge_before3week,next_winningscore_comp12,next_impr,next_real_cst_bdgt,next_real_aBln,next_feacomp_ctr,next_real_expose,next_real_revenue,next_revenue_before2week,next_ratioctr,next_real_price,next_lowerbid,next_ratio_aimcpc,next_cpc_before4week,next_order_num_before3week,next_histctr,next_mt_butie_before2week,next_mt_butie_before3week,next_real_budget,next_order_num_before4week,next_cpc,next_order_num_before1week,next_revenue_before1week,next_mt_butie,next_cpc_before2week,next_cpc_before3week,next_aimcpc,next_mt_butie_before1week,next_real_csm_bln,next_click_before3week,next_charge_before1week,next_click_before4week,next_click_before2week,next_real_click,next_winningscore_qs,next_cityid,next_ratio_click,next_prod,next_ratioctr_exp,next_upperk,next_charge,next_revenue_before4week,next_ctr,next_order_num,next_impr_before4week,next_impr_before2week,next_mt_butie_before4week,next_real_order,next_ratio_cpc,next_impr_before3week,next_charge_before4week,next_real_bln".toLowerCase() //RL 下一个状态特征
var cur_state_dynamic_fea="ctr,histctr,final_charge,aimcpc,impr,cpc,revenue,charge,click,order_num,mt_butie,impr_before1week,click_before1week,cpc_before1week,order_num_before1week,revenue_before1week,mt_butie_before1week,charge_before1week,impr_before2week,click_before2week,cpc_before2week,revenue_before2week,charge_before2week,order_num_before2week,mt_butie_before2week,impr_before3week,cpc_before3week,click_before3week,order_num_before3week,mt_butie_before3week,revenue_before3week,charge_before3week,impr_before4week,charge_before4week,revenue_before4week,mt_butie_before4week,cpc_before4week,click_before4week,order_num_before4week,budget,goal".toLowerCase() //RL 状态特征
var next_state_dynamic_fea="next_ctr,next_histctr,next_final_charge,next_aimcpc,next_impr,next_cpc,next_revenue,next_charge,next_click,next_order_num,next_mt_butie,next_impr_before1week,next_click_before1week,next_cpc_before1week,next_order_num_before1week,next_revenue_before1week,next_mt_butie_before1week,next_charge_before1week,next_impr_before2week,next_click_before2week,next_cpc_before2week,next_revenue_before2week,next_charge_before2week,next_order_num_before2week,next_mt_butie_before2week,next_impr_before3week,next_cpc_before3week,next_click_before3week,next_order_num_before3week,next_mt_butie_before3week,next_revenue_before3week,next_charge_before3week,next_impr_before4week,next_charge_before4week,next_revenue_before4week,next_mt_butie_before4week,next_cpc_before4week,next_click_before4week,next_order_num_before4week,next_budget,next_goal".toLowerCase() //RL 下一个状态特征

var reward="reward"//RL reward 字段名字
var poi_id="poi_id"// pv 后续会去掉，修改为key
var pvid="pvid"// pv 后续会去掉，修改为key
var action="action"//RL action 字段名字
var is_terminal="is_terminal" //RL terminal 字段
var path_prefix=s"viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/${user}/${train_table_name}" //模型和tf recoder位置
var cur_date=befor1day
// var cur_date="20221117"

var quote="\""
var start_date=befor1day //训练表的分区
var end_date=befor1day //训练表的分区
// var start_date="20221009" //训练表的分区
// var end_date="20221009" //训练表的分区

var use_adaptive="False"
var use_batch_loss="False"
var lambda_alpha="1e-1_1e-1_1e-1_1e-1"
var lambda_update_interval="100"
var lambda_budgets_target= "1_1_1_1"
var constraint_target="1_1_1_1"
var constraint_loss_weight="0.01_0.01_0.01_0.01"



var parameter_servers=1 //ps的数量
var workers=3 //work的数量
var account=user //AFO框架启动消息通知对象
var queue="root.zw05_training_cluster.hadoop-waimaiadrd.cpu_job"
// 模型配置区
var batch_size="256" // batch size
// var batch_size="128"
var epoch="3"
var learning_rate="0.00001"
var deep_layers="128,64,64" //网络设置
var gamma="1.0"
// var num_action="20" //action的个数
var num_action="11" //action的个数
var update_interval="300"
var embed_dim="8" //离散特征的embedding维度

var bit="22"//离散特征的空间
var ext_is_predict_serving="0"
var use_bn="False"
var model_name="REM" //是否使用REM
var i_loss_weight="0" //使用BCQ时iloss的权重
var i_regularization_weight="0" 
var q_loss_weight="1" //使用BCQ时qloss的权重
var num_heads="20" //REM head 个数
var threshold="0.3" // BCQ阈值
var use_cate_feature="True"
var use_dense_hashtable="True" //是否使用dense_hashtable
// var cur_state_cate_fea="poi_id,rn" //RL 状态特征
// var next_state_cate_fea="next_poi_id,next_rn" //RL 下一个状态特征
var cur_state_cate_fea="poi_id" //RL 状态特征
var next_state_cate_fea="poi_id" //RL 下一个状态特征
var model_file="udm/tangbo17/rbcq.py" //用户自定义模型代码文件
var readFromHive="True"
var file_path="rl_easy_go_high" // 模型的文件目录
var mod="local" //mod=local 代表启动workbench本地调试，其他代表提交到服务器
var code_branch="wanghao"

In [ ]:

var task="train" //任务类型 train,predict
var with_chief="true"
epoch="1"
mod="local1" 
var train_cmd=s"""
    if [[ ! -d "/opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter" ]]; 
    then
        echo "文件夹不存在"
        rm -rf /opt/meituan/dolphinfs_${user}/${train_table_name}
        mkdir /opt/meituan/dolphinfs_${user}/${train_table_name}
        cd /opt/meituan/dolphinfs_${user}/${train_table_name}
        git clone -b wanghao ssh://git@git.sankuai.com/~tangbo17/mtjupter.git
    else
        echo "文件夹存在"
    fi

    cd /opt/meituan/dolphinfs_${user}/${train_table_name}/mtjupter
    git pull
    ls
    cd OS/wanghao/${file_path}
    ls
    echo "开始train model"
    sh run.sh ${train_table_name} ${predict_table_name} ${model_name} ${batch_size} ${epoch} ${learning_rate} ${deep_layers} ${gamma} ${num_action} ${update_interval} ${embed_dim} ${bit} ${ext_is_predict_serving} ${use_bn} ${i_loss_weight} ${i_regularization_weight} ${q_loss_weight} ${num_heads} ${threshold} ${path_prefix} ${cur_date} ${cur_date} ${cur_state_dynamic_fea} ${next_state_dynamic_fea} ${task} ${parameter_servers} ${workers} ${account} ${queue} ${with_chief} ${use_dense_hashtable} ${cur_state_cate_fea} ${next_state_cate_fea} ${use_cate_feature} ${readFromHive} ${file_path} ${mod} ${train_table_name}/${trainDt} ${use_adaptive} ${use_batch_loss} ${lambda_alpha} ${lambda_update_interval} ${lambda_budgets_target} ${constraint_target} ${constraint_loss_weight}
    """
executeShell(train_cmd)

In [ ]:
executeShell(s"""
hope exec --jobname hadoop-hmart-waimaiad.spark.bid_framework --args="20221118 gen /ghnn01/user/hadoop-waimai/maoshangqin/evaFl_os/  viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/${train_table_name}"
""")

In [ ]:
//评估
executeShell(s"""
hope exec --jobname hadoop-hmart-waimaiad.spark.bid_framework --args="20221118 load_eval_rlk /ghnn01/user/hadoop-waimai/maoshangqin/evaFl_os/ viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/wanghao251/${train_table_name}/predict_result/${cur_date}/${train_table_name}_{model_name}"
""")

In [ ]:
val predict_rdd = spark.read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat")
      .option("sep", ",")
      .option("inferSchema", "true")
      .option("header", "true")
      .load("viewfs://hadoop-meituan/user/hadoop-hmart-waimaiad/tangbo17/mart_waimaiad.AFO_RL_1663158854324/predict_result/20220913/mart_waimaiad.AFO_RL_1663158854324_REM/predict_result_0.csv")